# The NormData class

A key component of the PCNtoolkit is the NormData object. It is a container for the data that will be used to fit the normative model. The NormData object keeps track of the all the dimensions of your data, the features and response variables, batch effects, preprocessing steps, and more. 

In [1]:
from pcntoolkit import NormData

## Creating a NormData object

There are currently two easy ways to create a NormData object. 
1. Load from a pandas dataframe
2. Load from numpy arrays 

Here are examples of both. 


In [2]:
# Creating a NormData object from a pandas dataframe
import pandas as pd

# Download an example dataset:
data = pd.read_csv(
    "https://raw.githubusercontent.com/predictive-clinical-neuroscience/PCNtoolkit-demo/refs/heads/main/data/fcon1000.csv"
)

# specify the column names to use
covariates = ["age"]
batch_effects = ["sex", "site"]
response_vars = ["WM-hypointensities", "Left-Lateral-Ventricle", "Brain-Stem"]

# create a NormData object
norm_data = NormData.from_dataframe(
    name="fcon1000", dataframe=data, covariates=covariates, batch_effects=batch_effects, response_vars=response_vars
)
norm_data.coords

Process: 91027 - 2025-06-12 15:20:31 - Dataset "fcon1000" created.
    - 1078 observations
    - 1078 unique subjects
    - 1 covariates
    - 3 response variables
    - 2 batch effects:
    	sex (2)
	site (23)
    


/opt/anaconda3/envs/uv_refactor/lib/python3.12/site-packages/pcntoolkit/util/output.py:216: UserWarning: Process: 91027 - 2025-06-12 15:20:31 - remove_Nan is set to False. Ensure your data does not contain NaNs in critical columns, or handle them appropriately.
  warnings.warn(message)


Coordinates:
  * observations       (observations) int64 9kB 0 1 2 3 ... 1074 1075 1076 1077
  * response_vars      (response_vars) <U22 264B 'WM-hypointensities' ... 'Br...
  * covariates         (covariates) <U3 12B 'age'
  * batch_effect_dims  (batch_effect_dims) <U4 32B 'sex' 'site'

In [3]:
# Creating a NormData object from numpy arrays
import numpy as np
from pcntoolkit import NormData

# synthesize some data
X = np.random.randn(100, 10)
Y = np.random.randn(100, 10)
batch_effects = np.random.randint(0, 2, 100)
subject_ids = np.arange(100)

# Create a NormData object
np_norm_data = NormData.from_ndarrays("fcon1000", X=X, Y=Y, batch_effects=batch_effects, subject_ids=subject_ids)
np_norm_data.coords

Process: 91027 - 2025-06-12 15:20:31 - Dataset "fcon1000" created.
    - 100 observations
    - 100 unique subjects
    - 10 covariates
    - 10 response variables
    - 1 batch effects:
    	batch_effect_0 (2)
    


Coordinates:
  * observations       (observations) int64 800B 0 1 2 3 4 5 ... 95 96 97 98 99
  * covariates         (covariates) <U11 440B 'covariate_0' ... 'covariate_9'
  * response_vars      (response_vars) <U14 560B 'response_var_0' ... 'respon...
  * batch_effect_dims  (batch_effect_dims) <U14 56B 'batch_effect_0'

As you can see, it is very simple to create a NormData object. 

There is an important difference though: the coordinates of the NormData object that was created with `from_dataframe` have the name of the column in the dataframe, but the `from_ndarrays` method creates coordinates with generic names. This is why the from_dataframe method is favorable.


## Casting back to a pandas dataframe

The NormData object can be cast back to a pandas dataframe using the `to_dataframe` method. This will return a pandas dataframe with a columnar multi-index. 


In [21]:
df = norm_data.to_dataframe()
df.head()

X          Y                                           Y_harmonized  \
     age Brain-Stem Left-Lateral-Ventricle WM-hypointensities   Brain-Stem   
0  25.63    20663.2                 4049.4             1686.7      20663.2   
1  18.34    19954.0                 9312.6             1371.1      19954.0   
2  29.20    21645.2                 8972.6             1414.8      21645.2   
3  31.39    20790.6                 6798.6             1830.6      20790.6   
4  13.58    17692.6                 6112.5             1642.4      17692.6   

                                                     Z                         \
  Left-Lateral-Ventricle WM-hypointensities Brain-Stem Left-Lateral-Ventricle   
0                 4049.4             1686.7   0.019085              -0.531192   
1                 9312.6             1371.1  -0.257872               1.066176   
2                 8972.6             1414.8   0.402664               0.377463   
3                 6798.6             1830.6   0.068779              -0.235935   
4                 6112.5             1642.4  -1.141237               0.612944   

                      ...           centiles                            \
  WM-hypointensities  ... (Brain-Stem, 0.25) (WM-hypointensities, 0.5)   
0           0.811669  ...       18887.797687               1039.652128   
1           0.786804  ...       18887.546534                743.969366   
2           0.288927  ...       18887.918689               1184.451505   
3           0.699007  ...       18887.992269               1273.278014   
4           1.369500  ...       18887.379599                550.903529   

                                                                              \
  (Left-Lateral-Ventricle, 0.5) (Brain-Stem, 0.5) (WM-hypointensities, 0.75)   
0                   6405.764862      20614.346287                1577.343220   
1                   4583.764377      20614.099824                1281.578935   
2                   7298.020243      20614.466982                1722.192384   
3                   7845.370183      20614.541023                1811.052643   
4                   3394.090536      20613.938897                1088.474467   

                                                     \
  (Left-Lateral-Ventricle, 0.75) (Brain-Stem, 0.75)   
0                    9397.799830       22340.894887   
1                    7575.345493       22340.653115   
2                   10290.332331       22341.015276   
3                   10837.870113       22341.089776   
4                    6385.456500       22340.498194   

                                                                                
  (WM-hypointensities, 0.95) (Left-Lateral-Ventricle, 0.95) (Brain-Stem, 0.95)  
0                2350.899842                   13702.331738       24824.817574  
1                2055.018273                   11879.224460       24824.582550  
2                2495.820633                   14595.262920       24824.937523  
3                2584.729447                   15143.070944       24825.012684  
4                1861.858226                   10689.025937       24824.436272  

[5 rows x 28 columns]

## Inspecting the NormData 

So let's go over the attributes of the NormData object. Because it is a subclass of xarray.Dataset, it has all the attributes of a xarray.Dataset, but it has some additional attributes that are specific to normative modelling. 

### The data variables

The data variables of the NormData object are:
- `X`: The covariates
- `Y`: The response variables
- `batch_effects`: The batch effects
- `subjects`: The subject ids

And all these data variables are xarray.DataArrays, with corresponding dimensions, stored in the `data_vars` attribute of the NormData object.

In [4]:
norm_data.data_vars

Data variables:
    subjects       (observations) int64 9kB 0 1 2 3 4 ... 1074 1075 1076 1077
    Y              (observations, response_vars) float64 26kB 1.687e+03 ... 1...
    X              (observations, covariates) float64 9kB 25.63 18.34 ... 23.0
    batch_effects  (observations, batch_effect_dims) <U17 147kB '1' ... 'Sain...

### The coordinates

Because it is a subclass of xarray.Dataset, the NormData object also holds all the coordinates of the data, found under the `coords` attribute. 

The coordinates are:
- `observations`: The index of the observations
- `response_vars`: The names of the response variables
- `covariates`: The names of the covariates
- `batch_effect_dims`: The names of the batch effect dimensions

In [5]:
norm_data.coords

Coordinates:
  * observations       (observations) int64 9kB 0 1 2 3 ... 1074 1075 1076 1077
  * response_vars      (response_vars) <U22 264B 'WM-hypointensities' ... 'Br...
  * covariates         (covariates) <U3 12B 'age'
  * batch_effect_dims  (batch_effect_dims) <U4 32B 'sex' 'site'

### Indexing using the coordinates

Xarrays powerful indexing methods can also be used on NormData. 

#### Selecting a response variable

For example, to select the data for a specific response variable, you can use the `response_vars` coordinate:

```python
norm_data.sel(response_vars="WM-hypointensities")
```

This will return a new NormData object with only the data for the response variable "WM-hypointensities".

In [8]:
norm_data.sel(response_vars="WM-hypointensities")

<xarray.NormData> Size: 181kB
Dimensions:            (observations: 1078, covariates: 1, batch_effect_dims: 2)
Coordinates:
  * observations       (observations) int64 9kB 0 1 2 3 ... 1074 1075 1076 1077
    response_vars      <U22 88B 'WM-hypointensities'
  * covariates         (covariates) <U3 12B 'age'
  * batch_effect_dims  (batch_effect_dims) <U4 32B 'sex' 'site'
Data variables:
    subjects           (observations) int64 9kB 0 1 2 3 ... 1074 1075 1076 1077
    Y                  (observations) float64 9kB 1.687e+03 1.371e+03 ... 509.1
    X                  (observations, covariates) float64 9kB 25.63 ... 23.0
    batch_effects      (observations, batch_effect_dims) <U17 147kB '1' ... '...
Attributes:
    real_ids:                       False
    is_scaled:                      False
    name:                           fcon1000
    unique_batch_effects:           {'sex': ['0', '1'], 'site': ['AnnArbor_a'...
    batch_effect_counts:            {'sex': {'0': 589, '1': 489}, 'site': {'A...
    batch_effect_covariate_ranges:  {'sex': {'0': {'age': {'min': 7.88, 'max'...
    covariate_ranges:               {'age': {'min': 7.88, 'max': 85.0}}

#### Selecting a number of observations

But we can also filter out a slice of the data. For example, to select the first 10 observations, you can use the `observations` coordinate:

```python
norm_data.sel(observations=slice(0, 9))
```

This will return a new NormData object with only the first 10 observations.

In [7]:
norm_data.sel(observations=slice(0, 9))

<xarray.NormData> Size: 2kB
Dimensions:            (observations: 10, response_vars: 3, covariates: 1,
                        batch_effect_dims: 2)
Coordinates:
  * observations       (observations) int64 80B 0 1 2 3 4 5 6 7 8 9
  * response_vars      (response_vars) <U22 264B 'WM-hypointensities' ... 'Br...
  * covariates         (covariates) <U3 12B 'age'
  * batch_effect_dims  (batch_effect_dims) <U4 32B 'sex' 'site'
Data variables:
    subjects           (observations) int64 80B 0 1 2 3 4 5 6 7 8 9
    Y                  (observations, response_vars) float64 240B 1.687e+03 ....
    X                  (observations, covariates) float64 80B 25.63 ... 19.88
    batch_effects      (observations, batch_effect_dims) <U17 1kB '1' ... 'An...
Attributes:
    real_ids:                       False
    is_scaled:                      False
    name:                           fcon1000
    unique_batch_effects:           {'sex': ['0', '1'], 'site': ['AnnArbor_a'...
    batch_effect_counts:            {'sex': {'0': 589, '1': 489}, 'site': {'A...
    batch_effect_covariate_ranges:  {'sex': {'0': {'age': {'min': 7.88, 'max'...
    covariate_ranges:               {'age': {'min': 7.88, 'max': 85.0}}

## NormData with predictions

After fitting a model and predicting on NormData, the NormData object will have new attributes holding the predictions. 


Specifically, the NormData object will be extended with new data variables:

- `Z`: The predicted Z scores for each response variable
- `centiles`: The predicted centiles
- `logp`: The predicted log-p-values for each response variable
- `Yhat`: The predicted mean of the response variable
- `Y_harmonized`: The harmonized response variables
- `statistics`: An array of statistics for each response variable


And the following new coordinates:
- `centile`: The specific centile values
- `statistic`: The name of the computed statistics


In [12]:
from pcntoolkit import NormativeModel, BLR

# We create a very simple BLR model because it is fast to fit
model = NormativeModel(BLR())
model.fit(norm_data)  # Fitting on the data also makes predictions for that data

Process: 91027 - 2025-06-12 16:00:30 - Fitting models on 3 response variables.
Process: 91027 - 2025-06-12 16:00:30 - Fitting model for WM-hypointensities.
Process: 91027 - 2025-06-12 16:00:30 - Fitting model for Left-Lateral-Ventricle.
Process: 91027 - 2025-06-12 16:00:30 - Fitting model for Brain-Stem.
Process: 91027 - 2025-06-12 16:00:30 - Making predictions on 3 response variables.
Process: 91027 - 2025-06-12 16:00:30 - Computing z-scores for 3 response variables.
Process: 91027 - 2025-06-12 16:00:30 - Computing z-scores for WM-hypointensities.
Process: 91027 - 2025-06-12 16:00:31 - Computing z-scores for Brain-Stem.
Process: 91027 - 2025-06-12 16:00:31 - Computing z-scores for Left-Lateral-Ventricle.
Process: 91027 - 2025-06-12 16:00:31 - Computing centiles for 3 response variables.
Process: 91027 - 2025-06-12 16:00:31 - Computing centiles for WM-hypointensities.
Process: 91027 - 2025-06-12 16:00:31 - Computing centiles for Brain-Stem.
Process: 91027 - 2025-06-12 16:00:31 - Comput

/opt/anaconda3/envs/uv_refactor/lib/python3.12/site-packages/pcntoolkit/util/output.py:216: UserWarning: Process: 91027 - 2025-06-12 16:00:31 - remove_Nan is set to False. Ensure your data does not contain NaNs in critical columns, or handle them appropriately.
  warnings.warn(message)


Process: 91027 - 2025-06-12 16:00:31 - Dataset "centile" created.
    - 150 observations
    - 150 unique subjects
    - 1 covariates
    - 3 response variables
    - 2 batch effects:
    	sex (1)
	site (1)
    
Process: 91027 - 2025-06-12 16:00:31 - Computing centiles for 3 response variables.
Process: 91027 - 2025-06-12 16:00:31 - Computing centiles for WM-hypointensities.
Process: 91027 - 2025-06-12 16:00:31 - Computing centiles for Brain-Stem.
Process: 91027 - 2025-06-12 16:00:31 - Computing centiles for Left-Lateral-Ventricle.
Process: 91027 - 2025-06-12 16:00:32 - Harmonizing data on 3 response variables.
Process: 91027 - 2025-06-12 16:00:32 - Harmonizing data for WM-hypointensities.
Process: 91027 - 2025-06-12 16:00:32 - Harmonizing data for Brain-Stem.
Process: 91027 - 2025-06-12 16:00:32 - Harmonizing data for Left-Lateral-Ventricle.


/opt/anaconda3/envs/uv_refactor/lib/python3.12/site-packages/pcntoolkit/util/plotter.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_be_df["marker"] = ["Other data"] * len(non_be_df)
/opt/anaconda3/envs/uv_refactor/lib/python3.12/site-packages/pcntoolkit/util/plotter.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_be_df["marker"] = ["Other data"] * len(non_be_df)


Process: 91027 - 2025-06-12 16:00:32 - Saving model to:
	/Users/stijndeboer/.pcntoolkit/saves.


/opt/anaconda3/envs/uv_refactor/lib/python3.12/site-packages/pcntoolkit/util/plotter.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_be_df["marker"] = ["Other data"] * len(non_be_df)


In [13]:
norm_data.data_vars

Data variables:
    subjects       (observations) int64 9kB 0 1 2 3 4 ... 1074 1075 1076 1077
    Y              (observations, response_vars) float64 26kB 1.687e+03 ... 1...
    X              (observations, covariates) float64 9kB 25.63 18.34 ... 23.0
    batch_effects  (observations, batch_effect_dims) <U17 147kB '1' ... 'Sain...
    Z              (observations, response_vars) float64 26kB 0.8117 ... -0.2591
    centiles       (centile, observations, response_vars) float64 129kB -271....
    logp           (observations, response_vars) float64 26kB -7.929 ... -8.8
    Yhat           (observations, response_vars) float64 26kB 1.04e+03 ... 2....
    statistics     (response_vars, statistic) float64 240B 0.05581 ... 0.9964
    Y_harmonized   (observations, response_vars) float64 26kB 1.687e+03 ... 1...

In [14]:
norm_data.coords

Coordinates:
  * observations       (observations) int64 9kB 0 1 2 3 ... 1074 1075 1076 1077
  * response_vars      (response_vars) <U22 264B 'WM-hypointensities' ... 'Br...
  * covariates         (covariates) <U3 12B 'age'
  * batch_effect_dims  (batch_effect_dims) <U4 32B 'sex' 'site'
  * centile            (centile) float64 40B 0.05 0.25 0.5 0.75 0.95
  * statistic          (statistic) <U8 320B 'MACE' 'MAPE' ... 'SMSE' 'ShapiroW'

### Indexing of predicted data

All the indexing methods can still be used, and they will also slice through the newly added data variables. So for example, to select the first 10 observations, you can use:

```python
norm_data.sel(observations=slice(0, 9))
```

This will return a new NormData object with only the first 10 observations.

In [15]:
norm_data.sel(observations=slice(0, 9))


<xarray.NormData> Size: 5kB
Dimensions:            (observations: 10, response_vars: 3, covariates: 1,
                        batch_effect_dims: 2, centile: 5, statistic: 10)
Coordinates:
  * observations       (observations) int64 80B 0 1 2 3 4 5 6 7 8 9
  * response_vars      (response_vars) <U22 264B 'WM-hypointensities' ... 'Br...
  * covariates         (covariates) <U3 12B 'age'
  * batch_effect_dims  (batch_effect_dims) <U4 32B 'sex' 'site'
  * centile            (centile) float64 40B 0.05 0.25 0.5 0.75 0.95
  * statistic          (statistic) <U8 320B 'MACE' 'MAPE' ... 'SMSE' 'ShapiroW'
Data variables:
    subjects           (observations) int64 80B 0 1 2 3 4 5 6 7 8 9
    Y                  (observations, response_vars) float64 240B 1.687e+03 ....
    X                  (observations, covariates) float64 80B 25.63 ... 19.88
    batch_effects      (observations, batch_effect_dims) <U17 1kB '1' ... 'An...
    Z                  (observations, response_vars) float64 240B 0.8117 ... ...
    centiles           (centile, observations, response_vars) float64 1kB -27...
    logp               (observations, response_vars) float64 240B -7.929 ... ...
    Yhat               (observations, response_vars) float64 240B 1.04e+03 .....
    statistics         (response_vars, statistic) float64 240B 0.05581 ... 0....
    Y_harmonized       (observations, response_vars) float64 240B 1.687e+03 ....
Attributes:
    real_ids:                       False
    is_scaled:                      False
    name:                           fcon1000
    unique_batch_effects:           {'sex': ['0', '1'], 'site': ['AnnArbor_a'...
    batch_effect_counts:            {'sex': {'0': 589, '1': 489}, 'site': {'A...
    batch_effect_covariate_ranges:  {'sex': {'0': {'age': {'min': 7.88, 'max'...
    covariate_ranges:               {'age': {'min': 7.88, 'max': 85.0}}

Or, if we want to select only the WM-hypointensities, we can use:

```python
norm_data.sel(response_vars="WM-hypointensities")
```

This will return a new NormData object with only the WM-hypointensities.

In [23]:
norm_data.sel(response_vars="WM-hypointensities").statistics

<xarray.DataArray 'statistics' (statistic: 10)> Size: 80B
array([ 5.58070501e-02,  4.53956023e-01, -3.25480412e-04,  8.07756061e+00,
       -1.13582231e-03,  7.79664882e+02,  9.91346248e-03,  7.45091622e-01,
        1.00113582e+00,  8.64052887e-01])
Coordinates:
    response_vars  <U22 88B 'WM-hypointensities'
  * statistic      (statistic) <U8 320B 'MACE' 'MAPE' ... 'SMSE' 'ShapiroW'

Now we can use the to_dataframe method to cast that selection back to a pandas dataframe.

In [22]:
new_df = norm_data.sel(response_vars="WM-hypointensities").to_dataframe()
new_df.head()

X                  Y       Y_harmonized                  Z  \
     age WM-hypointensities WM-hypointensities WM-hypointensities   
0  25.63             1686.7             1686.7           0.811669   
1  18.34             1371.1             1371.1           0.786804   
2  29.20             1414.8             1414.8           0.288927   
3  31.39             1830.6             1830.6           0.699007   
4  13.58             1642.4             1642.4           1.369500   

  batch_effects             subjects                   centiles  \
            sex        site subjects (WM-hypointensities, 0.05)   
0             1  AnnArbor_a        0                -271.595586   
1             1  AnnArbor_a        1                -567.079541   
2             1  AnnArbor_a        2                -126.917623   
3             1  AnnArbor_a        3                 -38.173419   
4             1  AnnArbor_a        4                -760.051167   

                                                        \
  (WM-hypointensities, 0.25) (WM-hypointensities, 0.5)   
0                 501.961036               1039.652128   
1                 206.359797                743.969366   
2                 646.710626               1184.451505   
3                 735.503385               1273.278014   
4                  13.332592                550.903529   

                                                         
  (WM-hypointensities, 0.75) (WM-hypointensities, 0.95)  
0                1577.343220                2350.899842  
1                1281.578935                2055.018273  
2                1722.192384                2495.820633  
3                1811.052643                2584.729447  
4                1088.474467                1861.858226

This should give you a pretty good overview of how to work with NormData. Most of the functionality is built on top of xarray, so if you want to learn more about xarray, you can check out the [xarray documentation](https://docs.xarray.dev/en/stable/). However, the Xarray.DataSet class does not officially support being extended, so the API does not work completely as expected. 

If you have any suggestions for improvements, please let us know!

